**Install libraries**

In [1]:
# Install necessary libraries
!pip install pretty_midi tensorflow numpy matplotlib



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 64.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 kB 5.0 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592287 sha256=1e8f01b1f77cb352c1dc2c5824f3508f8dd9fa33ab5cb366668e90d9ea9600ba
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


**Import libraries**

In [2]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import matplotlib.pyplot as plt


**Load MAESTRO dataset**

In [3]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [4]:
dataset_folder = '/content/drive/MyDrive/archive/maestro-v3.0.0'  # Adjust the path if needed


In [5]:
import os

# List the contents of the folder
os.listdir(dataset_folder)


['2004',
 '2014',
 '2018',
 '2006',
 '2011',
 '2008',
 '2017',
 '2015',
 '2009',
 '2013',
 'LICENSE',
 'README',
 'maestro-v3.0.0.csv',
 'maestro-v3.0.0.json']

**Load and Explore the Dataset**

In [7]:
import os

# Path to the MAESTRO dataset - Updated to use the previously mounted Google Drive path
dataset_path = "/content/drive/MyDrive/archive/maestro-v3.0.0"
# This path matches the path set in 'dataset_folder' in ipython-input-4-2d3163458081

# List the years available in the dataset
years = os.listdir(dataset_path)
print("Available folders (years):", years)

# Check the content of a specific folder
example_year = "2004"
example_path = os.path.join(dataset_path, example_year)
print("Files in", example_year, ":", os.listdir(example_path))

Available folders (years): ['2004', '2014', '2018', '2006', '2011', '2008', '2017', '2015', '2009', '2013', 'LICENSE', 'README', 'maestro-v3.0.0.csv', 'maestro-v3.0.0.json']
Files in 2004 : ['MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi', 'MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_06_Track06_wav.midi', 'MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_10_Track10_wav.midi', 'MIDI-Unprocessed_SMF_05_R1_2004_02-03_ORIG_MID--AUDIO_05_R1_2004_06_Track06_wav.midi', 'MIDI-Unprocessed_SMF_07_R1_2004_01_ORIG_MID--AUDIO_07_R1_2004_12_Track12_wav.midi', 'MIDI-Unprocessed_SMF_05_R1_2004_01_ORIG_MID--AUDIO_05_R1_2004_03_Track03_wav.midi', 'MIDI-Unprocessed_SMF_07_R1_2004_01_ORIG_MID--AUDIO_07_R1_2004_02_Track02_wav.midi', 'MIDI-Unprocessed_SMF_07_R1_2004_01_ORIG_MID--AUDIO_07_R1_2004_06_Track06_wav.midi', 'MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_08_Track08_wav.midi', 'MIDI-Unprocessed_SMF_05_R1_2

** Preprocess the Dataset**

In [8]:
import pretty_midi
import numpy as np

def midi_to_notes(file_path):
    pm = pretty_midi.PrettyMIDI(file_path)
    notes = []
    for instrument in pm.instruments:
        if not instrument.is_drum:
            for note in instrument.notes:
                notes.append([note.start, note.end, note.pitch, note.velocity])
    return np.array(notes)

# Example: Preprocessing a MIDI file
example_midi = os.path.join(example_path, os.listdir(example_path)[0])
notes = midi_to_notes(example_midi)
print("Extracted notes from", example_midi, ":", notes[:5])  # Print first 5 notes


Extracted notes from /content/drive/MyDrive/archive/maestro-v3.0.0/2004/MIDI-Unprocessed_SMF_02_R1_2004_01-05_ORIG_MID--AUDIO_02_R1_2004_05_Track05_wav.midi : [[ 1.09270833  1.18958333 71.         60.        ]
 [ 1.27916667  1.496875   55.         44.        ]
 [ 1.46354167  1.63125    59.         55.        ]
 [ 1.63333333  1.753125   62.         52.        ]
 [ 1.28854167  1.79375    71.         54.        ]]


** Prepare Training Data**

In [9]:
from sklearn.preprocessing import LabelEncoder

# Extract pitches for tokenization
pitches = [note[2] for note in notes]
encoder = LabelEncoder()
encoded_pitches = encoder.fit_transform(pitches)

# Prepare input-output sequences
sequence_length = 50
inputs, outputs = [], []
for i in range(len(encoded_pitches) - sequence_length):
    inputs.append(encoded_pitches[i:i+sequence_length])
    outputs.append(encoded_pitches[i+sequence_length])

inputs = np.array(inputs)
outputs = np.array(outputs)
print("Input shape:", inputs.shape, "Output shape:", outputs.shape)


Input shape: (7844, 50) Output shape: (7844,)


 **Design the Music Generation Model**

In [10]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model = Sequential([
    LSTM(256, input_shape=(sequence_length, 1), return_sequences=True),
    Dropout(0.3),
    LSTM(256, return_sequences=False),
    Dropout(0.3),
    Dense(256, activation='relu'),
    Dense(len(encoder.classes_), activation='softmax')
])

model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                          │ (None, 50, 256)             │         264,192 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout (Dropout)                    │ (None, 50, 256)             │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_1 (LSTM)                        │ (None, 256)                 │         525,312 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_1 (Dropout)                  │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense (Dense)                        │ (None, 256)                 │          65,792 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_1 (Dense)                      │ (None, 44)                  │          11,308 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 866,604 (3.31 MB)

 Trainable params: 866,604 (3.31 MB)

 Non-trainable params: 0 (0.00 B)

.** Train the Model**

In [11]:
# Reshape inputs for the model
inputs = inputs.reshape((inputs.shape[0], inputs.shape[1], 1))

# Train the model
history = model.fit(inputs, outputs, epochs=50, batch_size=64)


Epoch 1/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 6s 12ms/step - accuracy: 0.0569 - loss: 3.4910
Epoch 2/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 13ms/step - accuracy: 0.0622 - loss: 3.3609
Epoch 3/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 2s 11ms/step - accuracy: 0.0695 - loss: 3.2900
Epoch 4/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0851 - loss: 3.2329
Epoch 5/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0907 - loss: 3.1820
Epoch 6/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - accuracy: 0.0914 - loss: 3.1489
Epoch 7/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0910 - loss: 3.1209
Epoch 8/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.0977 - loss: 3.0799
Epoch 9/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 3s 13ms/step - accuracy: 0.1081 - loss: 3.0460
Epoch 10/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1178 - loss: 3.0276
Epoch 11/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 1s 11ms/step - accuracy: 0.1167 - loss: 2.9872
Epoch 12/50
123/123 ━━━━━━━━━━━━━━━━━━━━ 

**Generate New Music**

In [15]:
import random

# Start with a random seed
seed = inputs[random.randint(0, len(inputs) - 1)]
generated = seed.tolist() # generated is a list of lists

# Generate notes
for _ in range(200):  # Generate 200 notes
    # Reshape the seed to match the model's input shape
    seed_reshaped = seed.reshape(1, sequence_length, 1)

    prediction = model.predict(seed_reshaped)
    next_note = np.argmax(prediction)

    # Append only the integer, not a list
    generated.extend([next_note])

    # Update seed, ensuring it maintains the correct shape
    seed = np.concatenate([seed[1:], [[next_note]]])

# Flatten the generated list before inverse_transform
generated_flat = []
for sublist in generated:
    if isinstance(sublist, list):  # If sublist is a list, extend
        generated_flat.extend(sublist)
    else:  # If sublist is a single element, append
        generated_flat.append(sublist)

# Convert generated notes back to pitches
generated_pitches = encoder.inverse_transform(generated_flat)
print("Generated notes:", generated_pitches)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 16ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
1/1 ━━━━━━━━

 **Convert Generated Notes to MIDI**

In [20]:
import random
import pretty_midi
import numpy as np

def notes_to_midi(notes, output_file):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)
    for i in range(len(notes) - 1):
        start = i * 0.5  # Example: fixed duration
        end = (i + 1) * 0.5

        # Ensure pitch is an integer within the valid range (0-127)
        pitch = int(round(notes[i]))  # Round and convert to int
        pitch = max(0, min(pitch, 127))  # Clamp to 0-127

        # Create the note with validated pitch
        note = pretty_midi.Note(velocity=100, pitch=pitch, start=start, end=end)
        instrument.notes.append(note)
    midi.instruments.append(instrument)
    midi.write(output_file)

# ... (rest of your code to generate 'generated_pitches') ...

# Save generated music
output_midi = "generated_music.mid"
notes_to_midi(generated_pitches, output_midi)
print("Generated music saved to", output_midi)

Generated music saved to generated_music.mid


**Save the trained model**

In [21]:
# Save the trained model
model.save("music_generator_model.h5")

# Load the saved model
from tensorflow.keras.models import load_model
model = load_model("music_generator_model.h5")


In [24]:
notes_to_midi(generated_pitches, "new_generated_music.mid")


**Download**

In [25]:
from google.colab import files
files.download("new_generated_music.mid")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Saving classes.npy**

In [31]:
# Assuming you have your encoder and data from preprocessing
import numpy as np

# Replace `your_actual_data` with the variable containing your data
# For example, if your data is in a list called 'notes', use 'notes'
your_actual_data = []  # This should be replaced with your real data
encoder.fit(your_actual_data)  # or encoder.fit_transform(your_actual_data)
np.save("classes.npy", encoder.classes_)
print("Classes saved as 'classes.npy'")

Classes saved as 'classes.npy'


**Checking if classes.npy file has been saved or not**

In [32]:
classes = np.load("classes.npy")
print(classes)  # Verify the classes


[]


**Check file exist in google colaab or not**

In [33]:
!ls


classes.npy  generated_music.mid       new_generated_music.mid
drive	     music_generator_model.h5  sample_data


**Download classes.npy**

In [34]:
from google.colab import files
files.download("classes.npy")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

**Download the trained model**

In [35]:
from google.colab import files

# Save the model if not already saved
model.save("music_generator_model.h5")

# Download the model to your local machine
files.download("music_generator_model.h5")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.1/57.1 MB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.1/320.1 kB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 9.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.1/11.1 MB 119.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 12.3 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


**GRADIO app testing**

In [40]:
# Install required libraries (if not already installed)
!pip install gradio pretty_midi tensorflow numpy scikit-learn

# Import necessary libraries
import numpy as np
import pretty_midi
from tensorflow.keras.models import load_model
import gradio as gr
from sklearn.preprocessing import LabelEncoder

# Load the trained model and the encoder
model = load_model("music_generator_model.h5")  # Ensure this file is uploaded
encoder = LabelEncoder()

# Load the classes from 'classes.npy'
try:
    encoder.classes_ = np.load("classes.npy")  # Ensure this file is uploaded
except Exception as e:
    print(f"Error loading 'classes.npy': {e}")
    encoder.classes_ = None  # This will raise an error if the classes.npy file is missing or incorrect

# Function to convert generated notes to MIDI
def notes_to_midi(notes, output_file):
    midi = pretty_midi.PrettyMIDI()
    instrument = pretty_midi.Instrument(program=0)  # Acoustic Grand Piano
    for i, pitch in enumerate(notes):
        start = i * 0.5  # Example: each note is 0.5 seconds long
        end = (i + 1) * 0.5
        note = pretty_midi.Note(velocity=100, pitch=pitch, start=start, end=end)
        instrument.notes.append(note)
    midi.instruments.append(instrument)
    midi.write(output_file)

# Function to generate music with input validation
def generate_music(seed_input, num_notes):
    try:
        # Convert the seed input to a list of integers
        seed_sequence = [int(n) for n in seed_input.split(",")]
    except ValueError:
        return "Error: Seed input must be a comma-separated list of integers."

    # Validate if all notes are in MIDI range (0-127)
    if not all(0 <= n <= 127 for n in seed_sequence):
        return "Error: All notes must be between 0 and 127 (MIDI range)."

    # Check if the number of notes is reasonable
    if num_notes < 10 or num_notes > 500:
        return "Error: Number of notes must be between 10 and 500."

    # Generate the music sequence
    generated = seed_sequence.copy()
    try:
        for _ in range(num_notes):
            prediction = model.predict(np.array([seed_sequence]))
            next_note = np.argmax(prediction)
            generated.append(next_note)
            seed_sequence = seed_sequence[1:] + [next_note]
    except Exception as e:
        return f"Error during model prediction: {e}"

    # If classes.npy isn't loaded, return an error message
    if encoder.classes_ is None:
        return "Error: 'classes.npy' file is missing or not loaded correctly."

    # Convert generated notes back to MIDI pitches
    generated_pitches = encoder.inverse_transform(generated)

    # Save the generated music as a MIDI file
    output_file = "generated_music.mid"
    try:
        notes_to_midi(generated_pitches, output_file)
    except Exception as e:
        return f"Error while saving MIDI file: {e}"

    return output_file

# Gradio Interface
interface = gr.Interface(
    fn=generate_music,
    inputs=[
        gr.Textbox(label="Seed Notes (comma-separated integers)", placeholder="60,62,64,65"),
        gr.Slider(label="Number of Notes to Generate", minimum=10, maximum=500, step=10, value=100),
    ],
    outputs=gr.File(label="Download Generated MIDI File"),
    title="AI Music Generator",
    description="Generate music by providing seed notes and letting the AI complete the melody. You can download the generated MIDI file."
)

# Launch the Gradio interface
interface.launch(share=True)  # share=True allows public access to the interface


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://b2d8e00d7b325ab291.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
